# db-manager

Structure:
```
├── Cargo.toml
├── diesel.toml
└── src
    ├── lib.rs
    ├── postgres
    │   ├── blockchain
    │   │   ├── chain_blocks
    │   │   │   ├── model.rs
    │   │   │   └── mod.rs
    │   │   ├── chain_transactions
    │   │   │   ├── model.rs
    │   │   │   └── mod.rs
    │   │   ├── mod.rs
    │   │   └── transaction_pool
    │   │       ├── model.rs
    │   │       └── mod.rs
    │   ├── db.rs
    │   ├── migrations
    │   │   ├── 00000000000000_diesel_initial_setup
    │   │   │   ├── down.sql
    │   │   │   └── up.sql
    │   │   ├── 2022-04-21-080701_users
    │   │   │   ├── down.sql
    │   │   │   └── up.sql
    │   │   ├── 2022-04-25-173148_chain_blocks
    │   │   │   ├── down.sql
    │   │   │   └── up.sql
    │   │   ├── 2022-04-25-195047_chain_transactions
    │   │   │   ├── down.sql
    │   │   │   └── up.sql
    │   │   └── 2022-04-25-195956_transaction_pool
    │   │       ├── down.sql
    │   │       └── up.sql
    │   ├── mod.rs
    │   ├── schema.rs
    │   └── users
    │       ├── model.rs
    │       ├── mod.rs
    │       └── tests.rs
    └── traits.rs
```

Поднимается бд postgres, для работы с которой используется diesel (библиотека, с помошью которой можно не писать наприямую запросы sql к бд). Для этого сначала создаются так называемы migrations, там задаются все таблицы которые будут нами в дальнейшем использоваться. Итак, используемые здесь таблицы /postgres/migrations:
1) users
```sql
CREATE TABLE users (
  id SERIAL PRIMARY KEY,
  username VARCHAR NOT NULL,
  nickname VARCHAR NOT NULL,
  signature VARCHAR NOT NULL,
  balance INT NOT NULL
)
```
2) chain_blocks
```sql
CREATE TABLE chain_blocks (
  number SERIAL PRIMARY KEY,
  hash VARCHAR NOT NULL,
  previous_block_hash VARCHAR NOT NULL,
  timestamp INT NOT NULL,
  transactions TEXT[] NOT NULL
)
```
3) chain_transactions
```sql
CREATE TABLE chain_transactions (
  id SERIAL PRIMARY KEY,
  hash VARCHAR NOT NULL,
  nonce INT NOT NULL,
  from_ VARCHAR NOT NULL,
  to_ VARCHAR NOT NULL,
  value INT NOT NULL,
  signature VARCHAR NOT NULL
)
```
4) transaction_pool
```sql
CREATE TABLE transaction_pool (
  id SERIAL PRIMARY KEY,
  hash VARCHAR NOT NULL,
  nonce INT NOT NULL,
  from_ VARCHAR NOT NULL,
  to_ VARCHAR NOT NULL,
  value INT NOT NULL,
  signature VARCHAR NOT NULL
)
```

модуль /postgres/schema.rs не создается вручную, его генерирует Diesel. Когда мы запустили программу установки Diesel, был создан файл diesel.toml, который указывает Diesel сохранить это файл. В файле описаны такие же таблицы как выше с помощью макроса *table!*.

src/traits.rs (определение трейтов, которые в дальнейшем будут реализованый для соотетствующих структур):
```rust
#[async_trait::async_trait]
pub trait UsersManager {
    type User;
    type Error;

    async fn get_all(&self) -> Result<Vec<Self::User>, Self::Error>;
    async fn find(&self, username: String) -> Result<Self::User, Self::Error>;
    async fn delete(&self, username: String) -> Result<(), Self::Error>;
    async fn update(&self, user: Self::User) -> Result<(), Self::Error>;
    async fn insert(&self, user: Self::User) -> Result<(), Self::Error>;
}

#[async_trait::async_trait]
pub trait BlockchainManager {
    type Transaction;
    type Block;
    type Error;

    async fn get_last_block(&self) -> Result<Self::Block, Self::Error>;
    async fn submit_transaction(&self, transaction: Self::Transaction) -> Result<(), Self::Error>;
    async fn build_next_block(&self) -> Result<Self::Block, Self::Error>;
    async fn submit_mined_block(&self, block: Self::Block) -> Result<Self::Block, Self::Error>;
}
```

## Модуль postgres/users.

В **model.rs** определены структуры:
```rust
pub struct User {
    pub id: i32,
    pub username: String,
    pub nickname: String,
    pub signature: String,
    pub balance: i32,
}
```
и
```rust
pub struct UserForm {
    pub username: String,
    pub nickname: String,
    pub signature: String,
    pub balance: i32,
}
```

В **mod.rs** определена структура:
```rust
pub struct UserInfo {
    pub username: String,
    pub nickname: String,
    pub signature: String,
    pub balance: i32,
}

pub struct UserData;
```

### model.rs

Реализован стандартный трейт *From* для User, который позволяет преобразовывать одни типы данных в другие.
UserForm и UserInfo. Возможные преобразования:
1) $UserInfo \rightarrow UserForm$;

2) $User \rightarrow UserInfo$.

Также здесь реализован интерфейс структуры **User** для работы с бд, найти User, удалить User,... .

### mod.rs

Для структуры **UserData** реализован трейт **UserManager**.

## Модуль postgres/blockchain

### mod.rs

Объявлены структуры:
```rust
pub struct Transaction {
    pub hash: String,
    pub nonce: i32,
    pub from_: String,
    pub to_: String,
    pub value: i32,
    pub signature: String,
}

pub struct Block {
    pub number: i32,
    pub hash: String,
    pub previous_block_hash: String,
    pub timestamp: i32,
    pub transactions: Vec<String>,
}

pub struct Blockchain;
```

Для структуры **Blockchain** реализован трейт **BlockchainManager**.

### chain_blocks

#### model.rs

Объявлена своя структура:
```rust
pub struct Block {
    pub hash: String,
    pub previous_block_hash: String,
    pub timestamp: i32,
    pub transactions: Vec<String>,
}
```
И реализован стандартный трейт **From**, который конвертирует основную структуру **Block** в эту.

Объявлена структура:
```rust
pub struct ChainBlock {
    pub number: i32,
    pub hash: String,
    pub previous_block_hash: String,
    pub timestamp: i32,
    pub transactions: Vec<String>,
}
```
И также реализован стандартный трейт **From**, конвертирующий эту структуру в основную **Block**.
Также реализован интерфейс для структуры **ChainBlocks**, которая работает с таблицей chain_block бд.

### chain_transactions

Объявлены структуры:
```rust
pub struct Transaction {
    pub hash: String,
    pub nonce: i32,
    pub from_: String,
    pub to_: String,
    pub value: i32,
    pub signature: String,
}

pub struct ChainTransaction {
    pub id: i32,
    pub hash: String,
    pub nonce: i32,
    pub from_: String,
    pub to_: String,
    pub value: i32,
    pub signature: String,
}
```
Реализован стандартный трейт **From**, конвертирующий основную структуру **Transaction** в эту **Transaction**.
Также реализован интерфейс для структуры **ChainTransaction**, которая работает с таблицей chain_transactions бд.

### transaction_pool

Объявлены структуры:
```rust
pub struct Transaction {
    pub hash: String,
    pub nonce: i32,
    pub from_: String,
    pub to_: String,
    pub value: i32,
    pub signature: String,
}

pub struct PoolTransaction {
    pub id: i32,
    pub hash: String,
    pub nonce: i32,
    pub from_: String,
    pub to_: String,
    pub value: i32,
    pub signature: String,
}
```
Реализован стандартный трейт **From**, конвертирующий основную структуру **Transaction** в эту **Transaction**.
И также реализован стандартный трейт **From**, конвертирующий основную структуру **Transaction** в **PoolTransaction**.
Также реализован интерфейс для структуры **PoolTransaction**, которая работает с таблицей transaction_pool бд.



# general-logic

Structure:
```
├── Cargo.toml
└── src
    ├── blockchain
    │   ├── main.rs
    │   ├── mining
    │   │   ├── main.rs
    │   │   ├── mod.rs
    │   │   └── work.rs
    │   ├── mod.rs
    │   └── transaction
    │       ├── create
    │       │   ├── amount.rs
    │       │   ├── begin.rs
    │       │   ├── mod.rs
    │       │   ├── receiver.rs
    │       │   ├── send.rs
    │       │   └── signature.rs
    │       ├── main.rs
    │       └── mod.rs
    ├── game
    │   └── mod.rs
    ├── global
    │   ├── main.rs
    │   └── mod.rs
    └── lib.rs

```

# opia-bot

Structure:
```
├── Cargo.toml
└── src
    ├── logic
    │   ├── blockchain
    │   │   ├── main.rs
    │   │   ├── mining
    │   │   │   ├── main.rs
    │   │   │   ├── mod.rs
    │   │   │   └── work.rs
    │   │   ├── mod.rs
    │   │   └── transaction
    │   │       ├── create
    │   │       │   ├── amount.rs
    │   │       │   ├── begin.rs
    │   │       │   ├── mod.rs
    │   │       │   ├── receiver.rs
    │   │       │   ├── send.rs
    │   │       │   └── signature.rs
    │   │       ├── main.rs
    │   │       └── mod.rs
    │   ├── game
    │   │   └── mod.rs
    │   ├── global
    │   │   ├── main.rs
    │   │   └── mod.rs
    │   └── mod.rs
    └── main.rs
```

# providers

Structure:
```
├── Cargo.toml
└── src
    ├── blockchain
    │   ├── mod.rs
    │   └── traits.rs
    ├── lib.rs
    └── users
        ├── mod.rs
        └── traits.rs
```